In [1]:
! pip install pandas

     |████████████████████████████████| 11.5 MB 2.6 MB/s eta 0:00:01
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
     |████████████████████████████████| 15.7 MB 1.8 MB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np

In [6]:
! gunzip -v ../../incident-insights/data/public/public.csv.gz ../../incident-insights/data/public/ 

../../incident-insights/data/public/public.csv.gz:	 85.4% -- replaced with ../../incident-insights/data/public/public.csv
gzip: ../../incident-insights/data/public/ is a directory -- ignored


In [8]:
df = pd.read_csv('../../incident-insights/data/public/public.csv')

/home/sharad/miniconda3/envs/unearthed-challenge-name/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
df.shape

(12354494, 9)

In [10]:
df.columns

Index(['Work_DateTime', 'TIME_TYPE', 'WORK_NO', 'WORK_DESC', 'FUNC_CAT',
       'TOT_BRK_TM', 'EmpNo_Anon', 'Incident_Number', 'incident'],
      dtype='object')

In [12]:
df.incident.value_counts()

False    12354004
True          490
Name: incident, dtype: int64

In [13]:
df['Incident_Number'].nunique()

483

**Total number of employees**

In [19]:
df.EmpNo_Anon.nunique()

2114

In [22]:
df[df.TOT_BRK_TM == df.TOT_BRK_TM.max()].shape

(1785, 9)

In [115]:
df[(df.EmpNo_Anon == 38482332.0)]

,Work_DateTime,TIME_TYPE,WORK_NO,WORK_DESC,FUNC_CAT,TOT_BRK_TM,EmpNo_Anon,Incident_Number,incident,weather,day_of_week
26057,2014-07-31 08:00:00,Normal Time,4702883,DESIGN CONNECTION ASSETS MY031873,Support,6.0,38482332.0,NaN,False,Winter,3
26892,2014-07-31 09:00:00,Normal Time,4702883,DESIGN CONNECTION ASSETS MY031873,Support,6.0,38482332.0,NaN,False,Winter,3
27768,2014-07-31 10:00:00,Normal Time,4702883,DESIGN CONNECTION ASSETS MY031873,Support,6.0,38482332.0,NaN,False,Winter,3
28644,2014-07-31 11:00:00,Normal Time,4702883,DESIGN CONNECTION ASSETS MY031873,Support,6.0,38482332.0,NaN,False,Winter,3
29519,2014-07-31 12:00:00,Normal Time,4702883,DESIGN CONNECTION ASSETS MY031873,Support,6.0,38482332.0,NaN,False,Winter,3


In [112]:
df.EmpNo_Anon.value_counts()

38478632.0    15593
40922307.0    14813
41032807.0    14474
40859157.0    14399
38513682.0    14349
              ...  
38482332.0        5
41182307.0        4
41002457.0        4
40859957.0        3
40926557.0        3
Name: EmpNo_Anon, Length: 2114, dtype: int64

**Type of domains**

In [25]:
df.FUNC_CAT.value_counts()

Operational         7756407
Network or Asset    3277805
Support             1320282
Name: FUNC_CAT, dtype: int64

**Different type of works**

In [27]:
df.WORK_DESC.nunique()

221125

In [31]:
df.WORK_DESC.value_counts()[:10].index

Index(['Training', 'Admin', 'TCS: Pole Distribution',
       'NC- Distribution Standard Jobs', 'TCS: Conductor', 'Safety',
       'Housekeeping', 'TCS: NP   No Power', 'Meeting/briefings',
       'CONSTRUCTION'],
      dtype='object')

In [32]:
df.WORK_NO.unique()

array(['ME007497', 'ME007499', 'ME007502', ..., '06211411', '4999190E',
       '06210965'], dtype=object)

In [38]:
work_desc_df = df.groupby('WORK_DESC').WORK_NO.nunique().reset_index()

In [41]:
work_desc_df.sort_values('WORK_NO', ascending=False).head(10)

,WORK_DESC,WORK_NO
201856,TCS: NP No Power,32234
201569,TCS: Conductor,20339
201913,TCS: Pole Distribution,13542
201892,TCS: PP Part Power,10021
201878,TCS: PB Pole Broken/Damaged,7573
201825,TCS: Mini Pillar,7450
201628,TCS: ES Electric Shock,7245
201808,TCS: MH Miscellaneous Hazard,6705
201943,TCS: RC Reconnection,6117
201806,TCS: MC Miscellaneous Non Hazard,5749


- Every work description(`WORK_DESC`) has one to many mapping to work numbers(`WORK_NO`)

In [30]:
df.TIME_TYPE.value_counts()

Normal Time    10923131
Overtime        1431363
Name: TIME_TYPE, dtype: int64

**Minimum and maximum Work Time in entire data**

In [48]:
pd.to_datetime(df.Work_DateTime, format='%Y-%m-%d %H:%M:%S').max(), pd.to_datetime(df.Work_DateTime, format='%Y-%m-%d %H:%M:%S').min() 

(Timestamp('2019-12-31 23:00:00'), Timestamp('2014-05-12 07:00:00'))

In [49]:
df.loc[:, 'Work_DateTime'] = pd.to_datetime(df.Work_DateTime, format='%Y-%m-%d %H:%M:%S')

**Daylight saving is not followed in Perth. Follow the article: https://www.timeanddate.com/time/change/australia/perth#:~:text=Perth%20currently%20observes%20Australian%20Western,not%20change%20in%20Perth%2C%20Australia.**

In [69]:
weather_dict = {'Summer': [12, 1, 2], 'Autumn': [3, 4, 5], 'Winter': [6, 7, 8], 'Spring': [9, 10, 11]}
#https://www.australia.com/en-in/facts-and-planning/weather-in-australia/perth-weather.html

In [70]:
df.loc[df.Work_DateTime.dt.month.isin(weather_dict['Summer']), 'weather'] = 'Summer'
df.loc[df.Work_DateTime.dt.month.isin(weather_dict['Autumn']), 'weather'] = 'Autumn'
df.loc[df.Work_DateTime.dt.month.isin(weather_dict['Winter']), 'weather'] = 'Winter'
df.loc[df.Work_DateTime.dt.month.isin(weather_dict['Spring']), 'weather'] = 'Spring'

In [110]:
df.weather.value_counts(1)

Spring    0.286175
Winter    0.255008
Autumn    0.233317
Summer    0.225499
Name: weather, dtype: float64

**Incidence rate by weather**

In [111]:
df.groupby('weather').agg({'incident': ['mean', 'sum', 'count']})

incident              
             mean  sum    count
weather                        
Autumn   0.000038  110  2882518
Spring   0.000048  170  3535548
Summer   0.000041  114  2785932
Winter   0.000030   96  3150496

**Incident rate by day of the week**

In [80]:
df.loc[:, 'day_of_week'] = df.Work_DateTime.dt.weekday

In [90]:
df.groupby('day_of_week').agg({'incident': ['mean', 'sum']})

incident     
                 mean  sum
day_of_week               
0            0.000028   50
1            0.000039   97
2            0.000042  106
3            0.000042  106
4            0.000040   91
5            0.000055   19
6            0.000052   21

**Incident rate by domain**

In [98]:
df.groupby('FUNC_CAT').agg({'incident': ['mean', 'count', 'sum']})

incident              
                      mean    count  sum
FUNC_CAT                                
Network or Asset  0.000020  3277805   64
Operational       0.000052  7756407  403
Support           0.000017  1320282   23

In [97]:
work_groupby_df = df.groupby(['FUNC_CAT', 'WORK_DESC']).agg({'incident': ['mean', 'sum']}).reset_index()
work_groupby_df.columns = ['domain', 'work_desc', 'incident_mean', 'incident_sum']
work_groupby_df.sort_values('incident_sum', ascending=False).head(20)

,domain,work_desc,incident_mean,incident_sum
227404,Operational,Training,0.000039,14
127873,Operational,Admin,0.000053,14
223071,Operational,Safety,0.000114,13
224323,Operational,TCS: Pole Distribution,0.000067,12
149775,Operational,Inactive due to restricted duties,0.000421,7
224288,Operational,TCS: PB Pole Broken/Damaged,0.000125,7
224008,Operational,TCS: DOFT Drop Out Fuse Trip,0.000189,6
224302,Operational,TCS: PP Part Power,0.000183,6
149400,Operational,Housekeeping,0.000058,6
223980,Operational,TCS: Conductor,0.000049,6


In [102]:
df[df.incident == True].Work_DateTime.dt.month.value_counts(1) * 100

11    14.693878
10    12.244898
8      9.591837
3      9.183673
2      8.979592
9      7.755102
5      7.551020
12     7.142857
1      7.142857
7      6.938776
4      5.714286
6      3.061224
Name: Work_DateTime, dtype: float64

**Auxiliary Data**

In [116]:
aux_df = pd.read_csv('../data/auxillary_incidents.csv')

In [118]:
aux_df.columns

Index(['Unnamed: 0', 'Incident_Number', 'Consequences', 'Suburb/Locality',
       'Risk_Score', 'Employee_Type', 'OBS_Contractor_Company', 'OBS_Person',
       'OBS_Injury_Classification', 'Sign_Off_Date', 'Close_this_Record?',
       'Incident_DateTime', 'All_Consequences_of_the_Incident', 'Status',
       'Bodily_Location', 'Nature', 'Mechanism', 'Agency'],
      dtype='object')

In [121]:
aux_df.Incident_Number.nunique(), aux_df.shape[0]

(2802, 2915)

In [123]:
aux_df.Consequences.value_counts()

Injury or Illness                                                                                             2720
Motor Vehicle Incident, Injury or Illness                                                                       91
Injury or Illness, Near Miss                                                                                    47
Injury or Illness, Public Electric Shock                                                                        14
Network Asset Failure - Western Power, Injury or Illness                                                        12
<Undefined>, Injury or Illness                                                                                   6
Unplanned Outage, Injury or Illness                                                                              5
Environmental, Injury or Illness                                                                                 4
Environmental, Injury or Illness, Near Miss                                     

In [126]:
aux_df['Suburb/Locality'].nunique()

116

In [128]:
aux_df.Risk_Score.value_counts()

Low            1468
<Undefined>     856
Medium          529
High             61
Extreme           1
Name: Risk_Score, dtype: int64

In [130]:
aux_df.Employee_Type.value_counts()

Employee               1753
Contractor             1042
External Contractor      16
Name: Employee_Type, dtype: int64

In [134]:
aux_df.OBS_Person.nunique()

1025

In [136]:
aux_df.OBS_Injury_Classification.value_counts()

Classification 2            811
Classification 5            605
Classification 6            428
Classification 7            406
Classification 4            316
Classification 8            139
Classification 3             94
Undefined Classification      9
Classification 1              2
Name: OBS_Injury_Classification, dtype: int64

In [142]:
aux_df['Bodily_Location'].value_counts()

Hands                            451
<Undefined>                      181
Ankle                            159
Knees                            144
Back                             144
                                ... 
Feet, Chest, Ankle, Lower leg      1
Eyes, Neck, Upper back             1
Hands, Ankle                       1
Legs, Lower arm                    1
Arm, Elbow, Wrist                  1
Name: Bodily_Location, Length: 184, dtype: int64

In [146]:
aux_df['Agency'].value_counts()

Outdoor environment                        631
<Undefined>                                383
Other agencies                             291
Other materials, substances or objects     225
Human agencies                             176
Non-powered equipment                      148
Indoor environment                         134
Road transport                             129
Non-powered hand tools                     122
Live animals                               112
Mobile plant                               105
Powered equipment, tools and appliances     98
Machinery and fixed plant                   88
Non-living animals                          47
Unspecified agencies                        38
Biological agencies                         29
Other transport                             18
Non-physical agencies                       17
Chemicals                                   16
Non-metallic substances                      4
Underground environment                      1
Name: Agency,

**What We Do?**

- Western Power is a Western Australian State Government owned corporation responsible for building, maintaining and operating an electricity network that connects our 2.3 million customers to traditional and renewable energy sources, delivering an essential service to our community.